In [1]:
import sqlite3
import pandas as pd
import os

"""This code is used to create the wide format of the two classifications producing the columns for
dummy variables. For the classification 1, it creates n binary columns corresponding to each detected word.
For the classification 2, it creates n binary columns corresponding"""

# Create a SQL connection to our SQLite database
con = sqlite3.connect("DATABASES/project.db")

cur = con.cursor()

# Classification 1 transformation

In [2]:

# # Return all results of query
# cur.execute('SELECT plot_id FROM plots WHERE plot_type="Control"')
# cur.fetchall()

# Load the data into a DataFrame
regex_df = pd.read_sql_query("SELECT * from classification_1_regex", con)
match_df = pd.read_sql_query("SELECT * from classification_1_psittaciformes_or_no", con)
#link_df = pd.read_sql_query("SELECT * from classification_1_reg_map_match", con)
#bird_or_no_df = pd.read_sql_query("SELECT * from classification_1_parse_bird_or_no", con)

match_df.index = match_df.id
regex_df.index = regex_df.id

In [3]:
match_df["word"]=match_df["mapping_match"]\
    .apply(lambda x : ";".join([regex_df.at[int(_),"word"] for _ in x.split(";") if (len(x)>0)]))


In [4]:
words_df=match_df["word"].str.get_dummies(sep=";").add_prefix("word_")

In [5]:
words_df=match_df[["ad_id"]].join(words_df)
# words_df.reset_index(drop=True, inplace=True)
# words_df.index=words_df.id
words_df

,ad_id,word_(amazona),word_african,word_africà,word_ailes,word_all-blue,word_amazon,word_amazona,word_amazone,word_ara,...,word_versicolor,word_vert,word_vicente,word_vincent,word_yaco,word_yellow-collared,word_yellow-crested,word_yellow-headed,word_yellow-naped,word_yellow-shouldered
id,,,,,,,,,,,,,,,,,,,,,
1,552731_us,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,552645_us,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,552633_us,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,552632_us,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,552631_us,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25942,1060_ph,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25943,925_ph,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25944,523_ph,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
words_df.to_sql("classification_1_analysis", con, if_exists="replace")

# Classification 2

In [7]:
class_2 = pd.read_sql_query("SELECT * from classification_2_matching_ads", con)
class_2.index = class_2.id
cites = pd.read_sql_query("SELECT * from mapping_cites", con)
cites.index = cites.id



In [8]:
class_2["species"]=class_2["ids_matching"]\
    .apply(lambda x : ";".join([cites.at[int(_),"scientific_name_cites"] for _ in x.split(";") if (x!="-1" and x!="-2" and x!="")]))
species_df=class_2["species"].str.get_dummies(sep=";").add_prefix("")


In [9]:
species_df=class_2[["ad_id"]].join(species_df)

species_df.to_sql("classification_2_analysis", con, if_exists="replace")

# Classification 3

In [10]:
class_3 = pd.read_sql_query("SELECT * from classification_3_matching_ads", con)
class_3.index = class_3.id
cites = pd.read_sql_query("SELECT * from mapping_cites", con)
cites.index = cites.id



In [11]:
class_3["species"]=class_3["ids_matching"]\
    .apply(lambda x : ";".join([cites.at[int(_),"scientific_name_cites"] for _ in x.split(";") if (x!="-1" and x!="-2" and x!="")]))
species_df=class_3["species"].str.get_dummies(sep=";").add_prefix("species_")


In [12]:
species_df=class_3[["ad_id"]].join(species_df)

species_df.to_sql("classification_3_analysis", con, if_exists="replace")

In [13]:
# Be sure to close the connection
con.close()